In [ ]:
from flask import Flask, json, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

In [ ]:
@app.route("/tickets")
def getTickets():
    tickets = [
        {"name": "Leafs"},
        {"name": "Raptors"},
        {"name": "TFC"},
        {"name": "argos"}
    ]
    return jsonify(tickets)

In [ ]:
app.run(port=2000)